# SQL/Python

# Motivation:  What if your data is in a SQL database, but your machine learning is built in Python?

## Objectives

- Learn how to connect to and run PostgreSQL queries from Python
- Understand psycopg2's cursors, executes, and commits
- Learn how to generate dynamic queries through string formatting

## Combining SQL and Python

Often you will find yourself working with data that are only accessible through SQL.  However, your machine learning capabilities are built in Python.  To resolve this issue, we can simply set up a connection from Python to the SQL database to bring the data to us.

## Why do we care?

- SQL based databases are extremely common in almost all industry environments
- Can leverage the benefit of SQL's structure and scalability, while maintaining the flexibility of Python
- Very useful for scaled data pipelines, pre-cleaning, data exploration
- Allows for dynamic query generation and hence automation

## psycopg2

- A Python library that allows for connections with PostgresSQL databases to easily query and retrieve data for analysis
- [Documentation - Includes Installation Instructions](http://initd.org/psycopg/docs/install.html)
- In addition to what's listed in the documentation, if you have the Anaconda distribution of Python:

```python 
conda install psycopg2 
```

(worked for me)

- There are similar packages for other flavors of SQL that work much the same way.  E.g., `mysql-connector-python` (MySQL), `sqlite` (SQLite), `pymongo` (MongoDB) 

## General Workflow

1. Establish connection to PostgreSQL database using psycopg2
2. Create a cursor
3. Use the cursor to execute SQL queries
4. Commit SQL actions
4. Close the cursor and connection

## Walkthrough 1: Creating a database from Python

### Connect to the database

- Connections must be established using an existing database, username, database IP/URL, and maybe passwords
- If you need to create a database, you can first connect to PostgreSQL using the dbname 'postgres' to initialize

In [1]:
import psycopg2 as pg2

conn = pg2.connect(dbname='postgres', user='ivan', host='localhost')

conn

<connection object at 0x1046f9180; dsn: 'user=ivan host=localhost dbname=postgres', closed: 0>

### Commits

- Data changes are not actually stored until you choose to commit
- You can choose to have automatic commit by using ` autocommit = True`
- When connecting directly to the PostgreSQL Server to initiate server level commands such as creating a database, you must use the `autocommit = True` option since Postgres does not have "temporary" transactions at the database level

In [2]:
conn.autocommit = True

### Instantiate the cursor

- A cursor is a control structure that enables traversal over the records in a database.  You can think of it as an iterator or pointer for SQL data retrieval
- Executes and fetches data
- When the cursor points at the resulting output of a query, it can only read each observation once.  If you choose to see a previously read observation, you must rerun the query
- Can be closed without closing the connection

In [3]:
cur = conn.cursor()

###  Create a database

In [4]:
cur.execute('DROP DATABASE IF EXISTS temp;')
cur.execute('CREATE DATABASE temp;')

### Disconnect from the cursor and database

- Cursors and connections must be closed using `.close()` or else PostgreSQL will lock certain operation on the database/tables to connection is severed

In [5]:
cur.close() # optional, closing the connection always closes any associated cursors

conn.close() # closing the connection

## Walkthrough 2: Let's use our new database

### Connect to the database

In [6]:
conn = pg2.connect(dbname='temp', user='ivan', host='localhost')
conn.autocommit = True

cur = conn.cursor()

### Create a new table

In [7]:
query = '''
CREATE TABLE logins (
    userid INTEGER, 
    tmstmp TIMESTAMP, 
    type VARCHAR(10)
);
'''

cur.execute(query)

### Insert data into new table

In [8]:
query = '''
COPY logins 
    FROM '/Users/ivan/GitHub/DSI_Lectures/sql-python/ivan_corneillet/logins_data/logins-01.csv' 
    DELIMITER ',' 
    CSV;
'''

cur.execute(query)

### Run a query to get 30 records from our data

In [9]:
query = '''
SELECT *
    FROM logins
    LIMIT 30;
'''

cur.execute(query)

### Let's look at our data one line at a time

There are a number of ways to grab results from the cursor:

- `cur.fetchone()` - returns the next result
- `cur.next()` - returns the next result
- `cur.fetchmany(n)` - returns the next n results
- `cur.fetchall()` - returns all results in the result set
- `for res in cur:` - iterates over all results in the `cursorcur.fetchall()`
  
Again, when results are returned from a cursor object, they are returned as a generator (e.g., it gives back the results lazily).  Therefore, each result in the result set can only be accessed once so if we want it again, we have to re-run the query

In [10]:
cur.fetchone()

(579, datetime.datetime(2013, 11, 20, 3, 20, 6), 'mobile')

In [11]:
cur.fetchone()

(823, datetime.datetime(2013, 11, 20, 3, 20, 49), 'web')

### Now, many lines at a time

In [12]:
# fetchmany(n) to get n rows

cur.fetchmany(10)

[(953, datetime.datetime(2013, 11, 20, 3, 28, 49), 'web'),
 (612, datetime.datetime(2013, 11, 20, 3, 36, 55), 'web'),
 (269, datetime.datetime(2013, 11, 20, 3, 43, 13), 'web'),
 (799, datetime.datetime(2013, 11, 20, 3, 56, 55), 'web'),
 (890, datetime.datetime(2013, 11, 20, 4, 2, 33), 'mobile'),
 (330, datetime.datetime(2013, 11, 20, 4, 54, 59), 'mobile'),
 (628, datetime.datetime(2013, 11, 20, 4, 57, 22), 'mobile'),
 (398, datetime.datetime(2013, 11, 20, 5, 3, 19), 'mobile'),
 (482, datetime.datetime(2013, 11, 20, 5, 4, 43), 'mobile'),
 (581, datetime.datetime(2013, 11, 20, 5, 12, 3), 'mobile')]

### Or everything at once

In [13]:
# fetchall() grabs all remaining rows

cur.fetchall()

[(370, datetime.datetime(2013, 11, 20, 5, 26, 46), 'mobile'),
 (230, datetime.datetime(2013, 11, 20, 5, 28, 29), 'web'),
 (596, datetime.datetime(2013, 11, 20, 5, 28, 36), 'web'),
 (274, datetime.datetime(2013, 11, 20, 5, 43, 8), 'mobile'),
 (581, datetime.datetime(2013, 11, 20, 5, 47, 10), 'web'),
 (417, datetime.datetime(2013, 11, 20, 5, 54, 37), 'mobile'),
 (185, datetime.datetime(2013, 11, 20, 5, 56, 22), 'mobile'),
 (371, datetime.datetime(2013, 11, 20, 5, 58, 35), 'mobile'),
 (133, datetime.datetime(2013, 11, 20, 5, 59, 7), 'web'),
 (621, datetime.datetime(2013, 11, 20, 6, 1, 46), 'web'),
 (306, datetime.datetime(2013, 11, 20, 6, 3, 23), 'mobile'),
 (509, datetime.datetime(2013, 11, 20, 6, 4, 43), 'web'),
 (505, datetime.datetime(2013, 11, 20, 6, 9, 52), 'web'),
 (678, datetime.datetime(2013, 11, 20, 6, 34, 18), 'web'),
 (889, datetime.datetime(2013, 11, 20, 6, 36, 32), 'mobile'),
 (202, datetime.datetime(2013, 11, 20, 6, 43, 33), 'mobile'),
 (614, datetime.datetime(2013, 11, 20,

## Dynamic Queries

- A Dynamic Query is a query that is generated based on context

### Example

We have 8 login `.csv` files that we need to insert into the logins table.  Instead of doing a `COPY FROM` query 8 times, we should utilize Python (or any future languages) to make this more efficient.  This is possible due to tokenized strings

### First let's get an idea of how many records we start with

In [14]:
cur.execute('SELECT COUNT(*) FROM logins;')

cur.fetchall()

[(10000L,)]

### Create a query template and determine file path for imports

- Use string formatting to generate a query for each approved file
- [WARNING: BEWARE OF SQL INJECTION](http://initd.org/psycopg/docs/usage.html)
- NEVER use + or % to reformat strings to be used with .execute

In [15]:
userid = 590
#userid = '590; SELECT * FROM logins WHERE userid = 589'
#userid = '590; DROP TABLE logins'

terribly_unsafe = 'SELECT * FROM logins WHERE userid = ' + str(userid)
print terribly_unsafe

SELECT * FROM logins WHERE userid = 590


In [16]:
cur.execute(terribly_unsafe)

cur.fetchall()

[(590, datetime.datetime(2013, 11, 22, 8, 17, 34), 'mobile'),
 (590, datetime.datetime(2013, 11, 30, 22, 32, 43), 'mobile'),
 (590, datetime.datetime(2013, 12, 1, 11, 44, 48), 'web'),
 (590, datetime.datetime(2013, 12, 3, 9, 17, 29), 'mobile'),
 (590, datetime.datetime(2013, 12, 6, 4, 32, 15), 'mobile'),
 (590, datetime.datetime(2013, 12, 8, 16, 13, 44), 'mobile'),
 (590, datetime.datetime(2013, 12, 13, 6, 38, 52), 'mobile'),
 (590, datetime.datetime(2013, 12, 15, 10, 8, 33), 'mobile'),
 (590, datetime.datetime(2013, 12, 17, 21, 1, 17), 'mobile')]

<img src='https://imgs.xkcd.com/comics/exploits_of_a_mom.png'>

(https://xkcd.com/327/)

### String formatting

In [17]:
query = '''
SELECT *
    FROM logins
    WHERE userid = %(userid)s
'''

cur.execute(query, {'userid': 590})

cur.fetchall()

[(590, datetime.datetime(2013, 11, 22, 8, 17, 34), 'mobile'),
 (590, datetime.datetime(2013, 11, 30, 22, 32, 43), 'mobile'),
 (590, datetime.datetime(2013, 12, 1, 11, 44, 48), 'web'),
 (590, datetime.datetime(2013, 12, 3, 9, 17, 29), 'mobile'),
 (590, datetime.datetime(2013, 12, 6, 4, 32, 15), 'mobile'),
 (590, datetime.datetime(2013, 12, 8, 16, 13, 44), 'mobile'),
 (590, datetime.datetime(2013, 12, 13, 6, 38, 52), 'mobile'),
 (590, datetime.datetime(2013, 12, 15, 10, 8, 33), 'mobile'),
 (590, datetime.datetime(2013, 12, 17, 21, 1, 17), 'mobile')]

In [18]:
cur.execute(query, {'userid': '590; SELECT * FROM logins WHERE userid = 589'})

cur.fetchall()

DataError: invalid input syntax for integer: "590; SELECT * FROM logins WHERE userid = 589"
LINE 4:     WHERE userid = '590; SELECT * FROM logins WHERE userid =...
                           ^


### Use string formatting to generate a query for each approved file

In [19]:
import os

folder_path = os.path.join(os.getcwd(), 'logins_data')

folder_path

'/Users/ivan/GitHub/DSI_Lectures/sql-python/ivan_corneillet/logins_data'

In [20]:
query = '''
COPY logins 
    FROM %(file_path)s
    DELIMITER ','
    CSV;
'''

In [21]:
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv') and file_name != 'logins-01.csv':
        path = os.path.join(folder_path, file_name)
        cur.execute(query, {'file_path': path})
        print '{0} inserted into table.'.format(file_name)

logins-02.csv inserted into table.
logins-03.csv inserted into table.
logins-04.csv inserted into table.
logins-05.csv inserted into table.
logins-06.csv inserted into table.
logins-07.csv inserted into table.
logins-08.csv inserted into table.


### Let's check the total number of records we have right now

In [22]:
cur.execute('SELECT COUNT(*) FROM logins;')

cur.fetchall()

[(78588L,)]

### Close your connection

In [23]:
conn.close()

# Key things to remember

- Connections must be established using an existing database, username, database IP/URL, and maybe passwords
- If you have no created databases, you can connect to PostgreSQL using the dbname 'postgres' to initialize db commands
- Data changes are not actually stored until you choose to commit.  This can be done either through `conn.commit()` or setting `autocommit = True`.  Until commited, all transactions are only temporary stored
- `autocommit = True` is necessary to do database commands like `CREATE DATABASE`.  This is because PostgreSQL does not have temporary transactions at the database level
- If you ever need to build similar pipelines for other forms of database, there are libraries such Pyodbc which operates essentially the same
- SQL connection databases utilize cursors for data traversal and retrieval.  This is kind of like an iterator in Python
- Cursor operations typically go like the following:
  - Execute a query
  - Fetch rows from query result if it is a `SELECT` query
  - Because it is iterative, previously fetched rows can only be fetched again by rerunning the query
  - Close cursor through `.close()`
- Cursors and Connections must be closed using `.close()` or else Postgres will lock certain operation on the database/tables to connection is severed

# Exercise

You're given a file called `playgolf.csv` in the `playgolf_data` folder.  The file is comma delimited and the first row is the header.  Without opening and looking at the file, create a table and insert the data.  Here is the header and first row:


|date|outlook|temp|humidity|windy|result|
|---|---|---|---|---|---|
|7/1/14|sunny|85|85|FALSE|Don't Play|

In [24]:
conn = pg2.connect(dbname='temp', user='ivan', host='localhost')

In [25]:
cur = conn.cursor()

In [26]:
query = '''
CREATE TABLE play_golf (
    date DATE,
    outlook VARCHAR(30),
    temp INTEGER, 
    humidity INTEGER,
    windy BOOLEAN,
    result VARCHAR(30)
);
'''

cur.execute(query)

In [27]:
query = '''
COPY play_golf 
    FROM %(file_path)s 
    DELIMITER ',' 
    HEADER 
    CSV;
'''

file_path = os.path.join(os.getcwd(), 'playgolf_data', 'playgolf.csv')
cur.execute(query, {'file_path': file_path})

In [28]:
query = '''
SELECT *
    FROM play_golf;
'''

cur.execute(query)

cur.fetchall()

[(datetime.date(2014, 7, 1), 'sunny', 85, 85, False, "Don't Play"),
 (datetime.date(2014, 7, 2), 'sunny', 80, 90, True, "Don't Play"),
 (datetime.date(2014, 7, 3), 'overcast', 83, 78, False, 'Play'),
 (datetime.date(2014, 7, 4), 'rain', 70, 96, False, 'Play'),
 (datetime.date(2014, 7, 5), 'rain', 68, 80, False, 'Play'),
 (datetime.date(2014, 7, 6), 'rain', 65, 70, True, "Don't Play"),
 (datetime.date(2014, 7, 7), 'overcast', 64, 65, True, 'Play'),
 (datetime.date(2014, 7, 8), 'sunny', 72, 95, False, "Don't Play"),
 (datetime.date(2014, 7, 9), 'sunny', 69, 70, False, 'Play'),
 (datetime.date(2014, 7, 10), 'rain', 75, 80, False, 'Play'),
 (datetime.date(2014, 7, 11), 'sunny', 75, 70, True, 'Play'),
 (datetime.date(2014, 7, 12), 'overcast', 72, 90, True, 'Play'),
 (datetime.date(2014, 7, 13), 'overcast', 81, 75, False, 'Play'),
 (datetime.date(2014, 7, 14), 'rain', 71, 80, True, "Don't Play")]

In [29]:
import pandas as pd

df = pd.read_sql(query, conn)

df

,date,outlook,temp,humidity,windy,result
0,2014-07-01,sunny,85,85,False,Don't Play
1,2014-07-02,sunny,80,90,True,Don't Play
2,2014-07-03,overcast,83,78,False,Play
3,2014-07-04,rain,70,96,False,Play
4,2014-07-05,rain,68,80,False,Play
5,2014-07-06,rain,65,70,True,Don't Play
6,2014-07-07,overcast,64,65,True,Play
7,2014-07-08,sunny,72,95,False,Don't Play
8,2014-07-09,sunny,69,70,False,Play
9,2014-07-10,rain,75,80,False,Play


In [30]:
conn.close()